# Рад са Mistral моделима

## Увод

Ова лекција обухвата:
- Истраживање различитих Mistral модела
- Разумевање намена и сценарија за сваки модел
- Примери кода који приказују јединствене карактеристике сваког модела.


## Мистрал модели

У овој лекцији ћемо истражити три различита Мистрал модела: **Mistral Large**, **Mistral Small** и **Mistral Nemo**.

Сваки од ових модела је доступан бесплатно на Github Model marketplace-у. Код у овом ноутбуку користиће ове моделе за извршавање кода. Више детаља о коришћењу Github модела за [прототиписање са AI моделима](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) можете пронаћи овде.


## Мистрал Ларџ 2 (2407)
Мистрал Ларџ 2 је тренутно водећи модел компаније Мистрал и намењен је за пословну употребу.

Овај модел представља унапређење у односу на оригинални Мистрал Ларџ, нудећи:
- Већи контекстуални прозор – 128k уместо 32k
- Боље резултате у задацима из математике и програмирања – просечна тачност 76,9% уместо 60,4%
- Побољшане мултијезичке перформансе – подржани језици су: енглески, француски, немачки, шпански, италијански, португалски, холандски, руски, кинески, јапански, корејски, арапски и хинди.

Захваљујући овим карактеристикама, Мистрал Ларџ се истиче у:
- *Генерисању уз помоћ претраживања (RAG)* – захваљујући већем контекстуалном прозору
- *Позивима функција* – овај модел има уграђену подршку за позивање функција, што омогућава интеграцију са спољним алатима и API-јима. Позиви могу бити извршени паралелно или један за другим, у одређеном редоследу.
- *Генерисању кода* – модел је посебно добар у генерисању кода на Python, Java, TypeScript и C++.


У овом примеру користимо Mistral Large 2 да бисмо применили RAG шаблон на текстуални документ. Питање је написано на корејском и тиче се активности аутора пре факултета.

Користи се Cohere Embeddings Model за креирање ембедингса текстуалног документа, као и самог питања. За овај пример користи се faiss Python пакет као складиште вектора.

Промпт који се шаље Mistral моделу садржи и питање и пронађене делове текста који су слични питању. Модел затим даје одговор на природном језику.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small
Mistral Small је још један модел из породице Mistral модела, који спада у премијум/ентерпрајз категорију. Као што име каже, овај модел је Мали Језички Модел (SLM). Предности коришћења Mistral Small су:
- Уштеда трошкова у поређењу са Mistral LLM моделима као што су Mistral Large и NeMo – 80% мања цена
- Ниска латенција – бржи одговор у поређењу са Mistral LLM моделима
- Флексибилност – може да се користи у различитим окружењима са мање ограничења у погледу потребних ресурса.

Mistral Small је одличан за:
- Задатке са текстом као што су сумирање, анализа сентимента и превођење.
- Апликације где се често шаљу захтеви, због исплативости
- Задатке са кодом који захтевају ниску латенцију, као што су преглед и предлози кода


## Поређење Mistral Small и Mistral Large

Да бисте видели разлике у кашњењу између Mistral Small и Large, покрените следеће ћелије.

Требало би да приметите разлику у времену одговора од 3-5 секунди. Обратите пажњу и на дужину и стил одговора на исти упит.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Мистрал НеМо

У поређењу са друга два модела о којима је било речи у овој лекцији, Мистрал НеМо је једини бесплатан модел са Apache2 лиценцом.

Сматра се надоградњом на претходни отворени LLM модел из Мистрала, Мистрал 7B.

Још неке карактеристике НеМо модела су:

- *Ефикаснија токенизација:* Овај модел користи Tekken токенизатор уместо чешће коришћеног tiktoken-а. То омогућава боље перформансе на више језика и у раду са кодом.

- *Фино подешавање:* Основни модел је доступан за фино подешавање. Ово пружа већу флексибилност у случајевима када је потребно додатно прилагођавање.

- *Нативно позивање функција* – Као и Мистрал Ларге, овај модел је трениран за позивање функција. То га чини јединственим као један од првих отворених модела који то омогућавају.


## Мистрал НеМо

У поређењу са друга два модела о којима је било речи у овој лекцији, Мистрал НеМо је једини бесплатан модел са Apache2 лиценцом.

Сматра се надоградњом на претходни отворени LLM модел из Мистрала, Мистрал 7B.

Још неке карактеристике НеМо модела су:

- *Ефикаснија токенизација:* Овај модел користи Tekken токенизатор уместо чешће коришћеног tiktoken-а. То омогућава боље перформансе на више језика и у раду са кодом.

- *Фино подешавање:* Основни модел је доступан за фино подешавање. Ово пружа већу флексибилност за случајеве где је фино подешавање потребно.

- *Нативно позивање функција* – Као и Мистрал Ларге, овај модел је трениран за позивање функција. То га чини јединственим као један од првих отворених модела који то омогућава.


### Поређење токенизатора

У овом примеру ћемо видети како Mistral NeMo обрађује токенизацију у поређењу са Mistral Large.

Оба примера користе исти упит, али треба да приметите да NeMo враћа мање токена у односу на Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Учење се не завршава овде, наставите своје путовање

Након што завршите ову лекцију, погледајте нашу [Збирку за учење о генеративној вештачкој интелигенцији](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) како бисте наставили да унапређујете своје знање о генеративној вештачкој интелигенцији!



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални људски превод. Не сносимо одговорност за било каква неспоразума или погрешна тумачења настала коришћењем овог превода.
